In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import nltk
tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')
import seaborn as sns
import time
import re
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
random.seed(123)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
df_train = pd.read_csv("A.csv")
df_dev = pd.read_csv("A.csv")
df_test = pd.read_csv("A.csv")
# pour appliquer MLP sur Les données B
# df_train = pd.read_csv("B.csv")
# df_dev = pd.read_csv("B.csv")
# df_test = pd.read_csv("B.csv")

In [ ]:
#création des embeddings W2V

In [ ]:
test_comments = df_test['commentaire'].values.tolist()
dev_comments = df_dev['commentaire'].values.tolist()
train_comments = df_train['commentaire'].values.tolist()

In [ ]:
tokenized_sentences = test_comments+dev_comments+train_comments

In [ ]:
from nltk.tokenize import word_tokenize

tokens = []
length = len(tokenized_sentences)
for i in range(length):
    tokens.append(word_tokenize(tokenized_sentences[i]))

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(tokens, vector_size=100, min_count=1,epochs = 10)

In [ ]:
model.save("word2vec_final.model")

In [ ]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
model_w2v = Word2Vec.load("word2vec_final.model")

In [ ]:
train_list = [' '.join(item.split()[:100]) for item in df_train['commentaire'].values.tolist() if item]
dev_list = [' '.join(item.split()[:100]) for item in df_dev['commentaire'].values.tolist() if item]

In [ ]:
df_train['commentaire'] = train_list

In [ ]:
df_dev['commentaire']=dev_list

In [ ]:
X_train = df_train['commentaire']
X_dev = df_dev['commentaire']
tokenizer = Tokenizer(num_words=len(list(X_train.unique())))
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train) 
X_dev = tokenizer.texts_to_sequences(X_dev)
X_train = pad_sequences(X_train, maxlen=100)
X_dev = pad_sequences(X_dev, maxlen=100)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    try:
        vector = model_w2v.wv[word]
    except:
        continue
        print(i)
    embedding_matrix[i] = vector

In [ ]:
y_train = np.array(df_train['note'])
y_dev = np.array(df_dev['note'])

In [ ]:
num_classes = 10

In [ ]:
model = Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,
                            100,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=True))
model.add(tf.keras.layers.Flatten(input_shape=(102, 100)))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

st = time.time()

#create callback
filepath = 'my_best_model_MLP_without_meta_data_A.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]


history=model.fit(X_train, y_train, validation_data=(X_dev, y_dev),epochs=20, batch_size=128, verbose=1,callbacks=callbacks)

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [ ]:
X_test = Df_test['commentaire']

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('my_best_model_MLP_without_meta_data_A.hdf5')

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100)

In [ ]:
yhat = model.predict(X_test)

In [ ]:
prediction = np.argmax(yhat,axis =1)

In [ ]:
notes = [] #génération du fichier pour la plateforme d'évaluation
with open("sortie_my_best_model_MLP_without_meta_data_A.txt", "w") as f:
    for i in range(len(df_test['review_id'])):
        notes.append((prediction[i]+1)/2)
        f.write(str(df_test['review_id'].iloc[i])+' '+str(notes[i]).replace('.',',')+'\n')